In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

# Lets read the data from silver :
from pyspark.sql import SparkSession

# Load Silver data
df_bank_silver = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/LH_Finance_Project_Prathamesh_Kodgirwar.Lakehouse/Files/Silver/bank_data_cleaned")
df_customer_silver = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/LH_Finance_Project_Prathamesh_Kodgirwar.Lakehouse/Files/Silver/customer_data_cleaned")
df_transaction_silver = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/LH_Finance_Project_Prathamesh_Kodgirwar.Lakehouse/Files/Silver/transaction_data_cleaned")

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 4, Finished, Available, Finished)

In [3]:
df_bank_silver.printSchema()
df_customer_silver.printSchema()
df_transaction_silver.printSchema()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 5, Finished, Available, Finished)

root
 |-- Branch_ID: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Firm_Revenue: double (nullable = true)
 |-- Expenses: double (nullable = true)
 |-- Profit_Margin: double (nullable = true)

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Customer_Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Bank_Name: string (nullable = true)
 |-- Branch_ID: integer (nullable = true)

root
 |-- Transaction_ID: string (nullable = true)
 |-- Customer_ID: integer (nullable = true)
 |-- Account_Type: string (nullable = true)
 |-- Total_Balance: double (nullable = true)
 |-- Transaction_Amount: double (nullable = true)
 |-- Investment_Amount: double (nullable = true)
 |-- Investment_Type: string (nullable = true)
 |-- Transaction_Date: date (nullable = true)



Fixing the mismatched data types oin the following :

In [4]:
from pyspark.sql.functions import col

# df_bank_silver = df_bank_silver.withColumn("Branch_ID", col("Branch_ID").cast("integer"))
# df_customer_silver = df_customer_silver.withColumn("Customer_ID", col("Customer_ID").cast("integer"))
df_transaction_silver=df_transaction_silver.withColumn("Transaction_ID", col("Transaction_ID").cast("integer"))

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 6, Finished, Available, Finished)

In [5]:
df_bank_silver.printSchema()
df_customer_silver.printSchema()
df_transaction_silver.printSchema()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 7, Finished, Available, Finished)

root
 |-- Branch_ID: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Firm_Revenue: double (nullable = true)
 |-- Expenses: double (nullable = true)
 |-- Profit_Margin: double (nullable = true)

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Customer_Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Bank_Name: string (nullable = true)
 |-- Branch_ID: integer (nullable = true)

root
 |-- Transaction_ID: integer (nullable = true)
 |-- Customer_ID: integer (nullable = true)
 |-- Account_Type: string (nullable = true)
 |-- Total_Balance: double (nullable = true)
 |-- Transaction_Amount: double (nullable = true)
 |-- Investment_Amount: double (nullable = true)
 |-- Investment_Type: string (nullable = true)
 |-- Transaction_Date: date (nullable = true)



Lets check similarly if data consists of null values or not :

In [6]:
from pyspark.sql.functions import col, sum

def count_nulls(df):
    return df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Check null values in each table
nulls_bank = count_nulls(df_bank_silver)
nulls_customer = count_nulls(df_customer_silver)
nulls_transaction = count_nulls(df_transaction_silver)

# Show the result
nulls_bank.show()
nulls_customer.show()
nulls_transaction.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 8, Finished, Available, Finished)

+---------+----+------+------------+--------+-------------+
|Branch_ID|City|Region|Firm_Revenue|Expenses|Profit_Margin|
+---------+----+------+------------+--------+-------------+
|        0|   0|     0|           0|       0|            0|
+---------+----+------+------------+--------+-------------+

+-----------+---+-------------+----+------+---------+---------+
|Customer_ID|Age|Customer_Type|City|Region|Bank_Name|Branch_ID|
+-----------+---+-------------+----+------+---------+---------+
|          0|  0|            0|   0|     0|        0|        0|
+-----------+---+-------------+----+------+---------+---------+

+--------------+-----------+------------+-------------+------------------+-----------------+---------------+----------------+
|Transaction_ID|Customer_ID|Account_Type|Total_Balance|Transaction_Amount|Investment_Amount|Investment_Type|Transaction_Date|
+--------------+-----------+------------+-------------+------------------+-----------------+---------------+----------------+


**Standardizing Data** 

Here changing the date to standard date format :

In [7]:
from pyspark.sql.functions import to_date

df_transaction_silver = df_transaction_silver.withColumn("Transaction_Date", to_date(col("Transaction_Date"), "yyyy-MM-dd"))

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 9, Finished, Available, Finished)

In [8]:
df_transaction_silver.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 10, Finished, Available, Finished)

+--------------+-----------+------------+-------------+------------------+-----------------+-----------------+----------------+
|Transaction_ID|Customer_ID|Account_Type|Total_Balance|Transaction_Amount|Investment_Amount|  Investment_Type|Transaction_Date|
+--------------+-----------+------------+-------------+------------------+-----------------+-----------------+----------------+
|        300008|     201643|    Business|      60911.0| 5224.002812996517|          45985.0|Recurring Deposit|      2025-02-04|
|        300021|     209684|    Business|      70335.0|             361.0|          47823.0|Recurring Deposit|      2024-08-08|
|        300041|     202102|    Business|      86047.0|            1011.0|          19917.0|Recurring Deposit|      2022-08-12|
|        300045|     201453|    Business|      41400.0|            3924.0|          29495.0|Recurring Deposit|      2023-09-06|
|        300057|     202042|    Business|      79085.0|            3989.0|          46476.0|Recurring De

**On close observation we can notice that floating point numbers are not in order we can define it.**

In [9]:
from pyspark.sql.functions import round

df_bank_silver = df_bank_silver.withColumn("Expenses", round(col("Expenses"), 2))
df_bank_silver = df_bank_silver.withColumn("Profit_Margin", round(col("Profit_Margin"), 2))
df_bank_silver = df_bank_silver.withColumn("Firm_Revenue", round(col("Firm_Revenue"), 2))

df_transaction_silver = df_transaction_silver.withColumn("Total_Balance", round(col("Total_Balance"), 2))
df_transaction_silver = df_transaction_silver.withColumn("Transaction_Amount", round(col("Transaction_Amount"), 2))
df_transaction_silver = df_transaction_silver.withColumn("Investment_Amount", round(col("Investment_Amount"), 2))

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 11, Finished, Available, Finished)

In [10]:
df_bank_silver.show()
df_transaction_silver.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 12, Finished, Available, Finished)

+---------+------+------+------------+--------+-------------+
|Branch_ID|  City|Region|Firm_Revenue|Expenses|Profit_Margin|
+---------+------+------+------------+--------+-------------+
|     1004|Nagpur|  West|    282701.0|148194.0|        -14.1|
|     1012|Nagpur|  West|    969777.0|347637.0|        -11.4|
|     1036|Nagpur|  West|    729537.0|225109.0|         7.71|
|     1054|Nagpur|  West|    967410.0|419776.0|       -46.92|
|     1069|Nagpur|  West|   514074.18|279148.0|        64.56|
|     1077|Nagpur|  West|    985286.0|152371.0|        -6.82|
|     1081|Nagpur|  West|    810727.0|234848.0|        63.59|
|     1084|Nagpur|  West|    604344.0|406488.0|         4.09|
|     1118|Nagpur|  West|    204248.0|208385.0|         83.7|
|     1162|Nagpur|  West|    505660.0|216164.0|        39.35|
|     1189|Nagpur|  West|     91013.0|229704.0|        29.68|
|     1223|Nagpur|  West|    297321.0|143804.0|        70.78|
|     1250|Nagpur|  West|    366272.0|145010.0|        -7.91|
|     12

**Converting all string columns to lowercase for uniformity.**

In [11]:
from pyspark.sql.functions import lower

df_customer_silver = df_customer_silver.withColumn("Customer_Type", lower(col("Customer_Type")))
df_customer_silver = df_customer_silver.withColumn("City", lower(col("City")))
df_customer_silver = df_customer_silver.withColumn("Region", lower(col("Region")))
df_customer_silver = df_customer_silver.withColumn("Bank_Name", lower(col("Bank_Name")))

df_bank_silver = df_bank_silver.withColumn("City", lower(col("City")))
df_bank_silver = df_bank_silver.withColumn("Region", lower(col("Region")))

df_transaction_silver = df_transaction_silver.withColumn("Account_Type", lower(col("Account_Type")))
df_transaction_silver = df_transaction_silver.withColumn("Investment_Type", lower(col("Investment_Type")))

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 13, Finished, Available, Finished)

In [12]:
df_bank_silver.show()
df_customer_silver.show()
df_transaction_silver.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 14, Finished, Available, Finished)

+---------+------+------+------------+--------+-------------+
|Branch_ID|  City|Region|Firm_Revenue|Expenses|Profit_Margin|
+---------+------+------+------------+--------+-------------+
|     1004|nagpur|  west|    282701.0|148194.0|        -14.1|
|     1012|nagpur|  west|    969777.0|347637.0|        -11.4|
|     1036|nagpur|  west|    729537.0|225109.0|         7.71|
|     1054|nagpur|  west|    967410.0|419776.0|       -46.92|
|     1069|nagpur|  west|   514074.18|279148.0|        64.56|
|     1077|nagpur|  west|    985286.0|152371.0|        -6.82|
|     1081|nagpur|  west|    810727.0|234848.0|        63.59|
|     1084|nagpur|  west|    604344.0|406488.0|         4.09|
|     1118|nagpur|  west|    204248.0|208385.0|         83.7|
|     1162|nagpur|  west|    505660.0|216164.0|        39.35|
|     1189|nagpur|  west|     91013.0|229704.0|        29.68|
|     1223|nagpur|  west|    297321.0|143804.0|        70.78|
|     1250|nagpur|  west|    366272.0|145010.0|        -7.91|
|     12

**Customer Financial Summary (Gold Table)
Calculate total transaction amount and investment amount per customer.**

In [13]:
from pyspark.sql.functions import sum

df_gold_customer = df_transaction_silver.groupBy("Customer_ID").agg(
    sum("Transaction_Amount").alias("Total_Spent"),
    sum("Investment_Amount").alias("Total_Invested")
)


StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 15, Finished, Available, Finished)

In [14]:
df_gold_customer.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 16, Finished, Available, Finished)

+-----------+-----------+--------------+
|Customer_ID|Total_Spent|Total_Invested|
+-----------+-----------+--------------+
|     209955|    10317.0|      103953.0|
|     206719|     2416.0|       22631.0|
|     205392|      119.0|       20675.0|
|     204877|     3649.0|       24313.0|
|     206378|     2957.0|       35931.0|
|     201946|     4876.0|       44179.0|
|     209029|     1003.0|        9014.0|
|     202641|     6219.0|       61471.0|
|     204780|     3325.0|       15867.0|
|     203429|     8291.0|       42464.0|
|     206817|     4111.0|       12971.0|
|     209326|      244.0|        8063.0|
|     206351|     2984.0|       34253.0|
|     205306|     5857.0|       55988.0|
|     204908|     2324.0|       14388.0|
|     207398|     4324.0|       14334.0|
|     202242|     4266.0|       48906.0|
|     207103|     2356.0|       30155.0|
|     203504|     2180.0|       19059.0|
|     204529|     2349.0|       26989.0|
+-----------+-----------+--------------+
only showing top

**Profitability Report by Bank->Branch (Gold Table)
Calculate profit margin percentage for each branch.**

In [15]:
from pyspark.sql.functions import (col, expr)

df_gold_branch = df_bank_silver.withColumn(
    "Profit_Percentage",
    expr("Profit_Margin / Firm_Revenue * 100")  # Converting to percentage
)

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 17, Finished, Available, Finished)

In [16]:
df_gold_branch.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 18, Finished, Available, Finished)

+---------+------+------+------------+--------+-------------+--------------------+
|Branch_ID|  City|Region|Firm_Revenue|Expenses|Profit_Margin|   Profit_Percentage|
+---------+------+------+------------+--------+-------------+--------------------+
|     1004|nagpur|  west|    282701.0|148194.0|        -14.1|-0.00498760174176...|
|     1012|nagpur|  west|    969777.0|347637.0|        -11.4|-0.00117552798220...|
|     1036|nagpur|  west|    729537.0|225109.0|         7.71|0.001056834677336...|
|     1054|nagpur|  west|    967410.0|419776.0|       -46.92|-0.00485006357180...|
|     1069|nagpur|  west|   514074.18|279148.0|        64.56|0.012558498853219977|
|     1077|nagpur|  west|    985286.0|152371.0|        -6.82|-6.92184807253934...|
|     1081|nagpur|  west|    810727.0|234848.0|        63.59|0.007843577431120463|
|     1084|nagpur|  west|    604344.0|406488.0|         4.09|6.767668744953205E-4|
|     1118|nagpur|  west|    204248.0|208385.0|         83.7|0.040979593435431436|
|   

In [17]:
from pyspark.sql.functions import col, expr, round

df_gold_branch = df_bank_silver.withColumn(
    "Profit_Percentage",
    round(expr("Profit_Margin / Firm_Revenue * 100"), 2)  # Rounding to 2 decimal places
)

df_gold_branch.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 19, Finished, Available, Finished)

+---------+------+------+------------+--------+-------------+-----------------+
|Branch_ID|  City|Region|Firm_Revenue|Expenses|Profit_Margin|Profit_Percentage|
+---------+------+------+------------+--------+-------------+-----------------+
|     1004|nagpur|  west|    282701.0|148194.0|        -14.1|              0.0|
|     1012|nagpur|  west|    969777.0|347637.0|        -11.4|              0.0|
|     1036|nagpur|  west|    729537.0|225109.0|         7.71|              0.0|
|     1054|nagpur|  west|    967410.0|419776.0|       -46.92|              0.0|
|     1069|nagpur|  west|   514074.18|279148.0|        64.56|             0.01|
|     1077|nagpur|  west|    985286.0|152371.0|        -6.82|              0.0|
|     1081|nagpur|  west|    810727.0|234848.0|        63.59|             0.01|
|     1084|nagpur|  west|    604344.0|406488.0|         4.09|              0.0|
|     1118|nagpur|  west|    204248.0|208385.0|         83.7|             0.04|
|     1162|nagpur|  west|    505660.0|21

**Bank Performance Analysis (Gold Table)
Analyze total customer transactions and investment per bank.**

In [18]:
df_gold_bank = df_transaction_silver.join(df_customer_silver, "Customer_ID", "inner").groupBy("Bank_Name").agg(
    sum("Transaction_Amount").alias("Total_Transactions"),
    sum("Investment_Amount").alias("Total_Investments")
)


StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 20, Finished, Available, Finished)

In [19]:
df_gold_bank.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 21, Finished, Available, Finished)

+---------+--------------------+-----------------+
|Bank_Name|  Total_Transactions|Total_Investments|
+---------+--------------------+-----------------+
|hdfc bank|2.5427080480000023E7|     2.55502484E8|
+---------+--------------------+-----------------+



**Score customers based on risky behavior like:**

**Overdrafts**

**High volume of transactions**

**Large transactions**

**Suspicious/frequent withdrawals**

In [22]:
from pyspark.sql.functions import when, lit, count

df_customer_behavior = df_transaction_silver.groupBy("customer_id").agg(
    sum("transaction_amount").alias("total_txn"),
    count("*").alias("txn_count")
)

df_gold_risk_scores = df_customer_behavior.withColumn(
    "risk_score",
    when(col("total_txn") > 500000, lit("High"))
    .when(col("txn_count") > 100, lit("Medium"))
    .otherwise(lit("Low"))
)


StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 24, Finished, Available, Finished)

In [23]:
df_gold_risk_scores.show()

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 25, Finished, Available, Finished)

+-----------+---------+---------+----------+
|customer_id|total_txn|txn_count|risk_score|
+-----------+---------+---------+----------+
|     209955|  10317.0|        3|       Low|
|     206719|   2416.0|        1|       Low|
|     205392|    119.0|        1|       Low|
|     204877|   3649.0|        1|       Low|
|     206378|   2957.0|        1|       Low|
|     201946|   4876.0|        1|       Low|
|     209029|   1003.0|        1|       Low|
|     202641|   6219.0|        2|       Low|
|     204780|   3325.0|        1|       Low|
|     203429|   8291.0|        2|       Low|
|     206817|   4111.0|        1|       Low|
|     209326|    244.0|        1|       Low|
|     206351|   2984.0|        1|       Low|
|     205306|   5857.0|        2|       Low|
|     204908|   2324.0|        1|       Low|
|     207398|   4324.0|        1|       Low|
|     202242|   4266.0|        1|       Low|
|     207103|   2356.0|        1|       Low|
|     203504|   2180.0|        1|       Low|
|     2045

In [24]:
# Saving Customer Summary
df_gold_customer.write.format("delta").mode("overwrite").save("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/LH_Finance_Project_Prathamesh_Kodgirwar.Lakehouse/Files/Gold/gold_customer_summary")

# Saving Branch Profitability
df_gold_branch.write.format("delta").mode("overwrite").save("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/LH_Finance_Project_Prathamesh_Kodgirwar.Lakehouse/Files/Gold/gold_branch_profitability")

# Saving Bank Performance
df_gold_bank.write.format("delta").mode("overwrite").save("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/LH_Finance_Project_Prathamesh_Kodgirwar.Lakehouse/Files/Gold/gold_bank_performance")

# Saving Risk Scores
df_gold_risk_scores.write.format("delta").mode("overwrite").save("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/LH_Finance_Project_Prathamesh_Kodgirwar.Lakehouse/Files/Gold/gold_risk_scores")

StatementMeta(, 0ca75b46-254a-445d-b5a7-6d542821a960, 26, Finished, Available, Finished)